<a href="https://colab.research.google.com/github/ronglu-stanford/RL_reference_public/blob/main/1_parallel_quicksort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import time
import sys
from multiprocessing import Process, Pipe

def quicksort(arr):
    """
    quicksort implementation, return a new sorted version
    of the input list.
    Faster quicksort in that it relies on built-in list
    comprehensions and concatenation.
    """
    if len(arr) <= 1:
        return arr
    pivot = arr.pop(len(arr)-1)

    return quicksort([x for x in arr if x < pivot]) \
        + [pivot] \
        + quicksort([x for x in arr if x >= pivot])

def quicksortParallel(arr, conn, procNum):
    """
    Partition the list, then quicksort the left and right
    sides in parallel.
    """

    print(f"Starting quicksort with list length {len(arr)}; procNum {procNum}")

    if procNum <= 0 or len(arr) <= 1:
        # In the case of len(arr) <= 1, quicksort will
        # immediately return anyway.
        conn.send(quicksort(arr))
        conn.close()
        return

    # Create two lists that can be sorted independently.
    pivot = arr.pop(len(arr)-1)

    leftSide = [x for x in arr if x < pivot]
    rightSide = [x for x in arr if x >= pivot]

    # Create a Pipe to communicate with the left subprocess
    pconnLeft, cconnLeft = Pipe()
    # Create a leftProc that executes quicksortParallel on
    # the left half-list.
    print(f"quicksort list length {len(arr)} calling left side {len(leftSide)}")
    leftProc = Process(target=quicksortParallel,
                       args=(leftSide, cconnLeft, procNum - 1))

    # Again, for the right.
    pconnRight, cconnRight = Pipe()
    print(f"quicksort list length {len(arr)} calling right side {len(rightSide)}")    
    rightProc = Process(target=quicksortParallel,
                        args=(rightSide, cconnRight, procNum - 1))

    # Start the two subprocesses.
    leftProc.start()
    rightProc.start()

    # Our answer is the concatenation of the subprocesses'
    # answers, with the pivot in between.
    conn.send(pconnLeft.recv() + [pivot] + pconnRight.recv())
    conn.close()

    # Join our subprocesses.
    leftProc.join()
    rightProc.join()


def isSorted(arr):
    """
    Return whether the argument arr is in non-decreasing order.
    """
    for i in range(1, len(arr)):
        if arr[i] < arr[i-1]:
            return False
    return True


In [ ]:
"""
This is the main method, where we:
- generate a random list.
- time a sequential quicksort on the list.
- time a parallel quicksort on the list.
- time Python's built-in sorted on the list.
"""
N = 1000000

# We want to sort the same list, so make a backup.
random.seed(3)
arrbck = [random.random() for x in range(N)]

# Sequential quicksort a copy of the list.
arr = list(arrbck)  # copy the list
start = time.time()  # start time
arr = quicksort(arr)  # quicksort the list
elapsed = time.time() - start  # stop time

if not isSorted(arr):
    print('quicksort did not sort the list. oops.')

print('Sequential quicksort: %f sec' % (elapsed))

# Parallel quicksort.
arr = list(arrbck)

start = time.time()
n = 3  # 2**n processes will be instantiated.

# Instantiate a Pipe so that we can receive the
# process's response.
pconn, cconn = Pipe()

# Instantiate a process that executes quicksortParallel
# on the entire list.
p = Process(target=quicksortParallel, args=(arr, cconn, n))
p.start()

arr = pconn.recv()
# Blocks until there is something (the sorted list)
# to receive.

p.join()
elapsed = time.time() - start

if not isSorted(arr):
    print('quicksortParallel did not sort the list. oops.')

print('Parallel quicksort: %f sec' % (elapsed))

# Built-in test.
# The underlying C code is the fastest, but this isn't the point here.
arr = list(arrbck)
start = time.time()
arr = sorted(arr)
elapsed = time.time() - start
print('Built-in sorted: %f sec' % (elapsed))

Sequential quicksort: 7.021569 sec
Starting quicksort with list length 1000000; procNum 3
quicksort list length 999999 calling left side 679563
quicksort list length 999999 calling right side 320436
Starting quicksort with list length 679563; procNum 2
Starting quicksort with list length 320436; procNum 2
quicksort list length 320435 calling left side 197069
quicksort list length 320435 calling right side 123366
Starting quicksort with list length 197069; procNum 1
Starting quicksort with list length 123366; procNum 1
quicksort list length 197068 calling left side 156677
quicksort list length 197068 calling right side 40391
Starting quicksort with list length 40391; procNum 0
Starting quicksort with list length 156677; procNum 0
quicksort list length 679562 calling left side 513335
quicksort list length 679562 calling right side 166227
quicksort list length 123365 calling left side 71946
quicksort list length 123365 calling right side 51419
Starting quicksort with list length 513335; p